In [1]:
import os

In [2]:
%pwd

'd:\\model deploy end to end\\NLP-text_classification\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\model deploy end to end\\NLP-text_classification'

In [5]:
import pandas as pd

In [6]:
data = pd.read_csv("artifacts/data_ingestion/SMSSpamCollection.tsv",sep='\t',names=['label','text'])
data.head()

,label,text
0,ham,I've been searching for the right words to tha...
1,spam,Free entry in 2 a wkly comp to win FA Cup fina...
2,ham,"Nah I don't think he goes to usf, he lives aro..."
3,ham,Even my brother is not like to speak with me. ...
4,ham,I HAVE A DATE ON SUNDAY WITH WILL!!


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5568 entries, 0 to 5567
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5568 non-null   object
 1   text    5568 non-null   object
dtypes: object(2)
memory usage: 87.1+ KB


In [8]:
data.isnull().sum()

label    0
text     0
dtype: int64

In [9]:
data.shape

(5568, 2)

In [10]:
data.columns

Index(['label', 'text'], dtype='object')

In [11]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [12]:
from src.NLPtc.constants import *
from src.NLPtc.utils.common import read_yaml, create_directories

In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.Columns

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [14]:
import os
from src.NLPtc import logger
from sklearn.model_selection import train_test_split

In [15]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir,sep='\t',names=['label','text'])
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                        
            logger.info(validation_status)
        
        except Exception as e:
            raise e
        
    def train_test_spliting(self):
        data = pd.read_csv(self.config.unzip_data_dir,sep='\t',names=['label','text'])
            
        # Split the data into training and test sets. (0.75, 0.25) split.
        x = data.drop('label',axis=1)
        y = data.label
        xtrain, xtest, ytrain, ytest = train_test_split(x,y,random_state=42)

        xtrain.to_csv(os.path.join(self.config.root_dir, "xtrain.csv"),index = False)
        xtest.to_csv(os.path.join(self.config.root_dir, "xtest.csv"),index = False)
        ytrain.to_csv(os.path.join(self.config.root_dir, "ytrain.csv"),index = False)
        ytest.to_csv(os.path.join(self.config.root_dir, "ytest.csv"),index = False)


        logger.info("Splited data into training and test sets")
        logger.info(xtrain.shape)
        logger.info(xtest.shape)
        logger.info(ytrain.shape)
        logger.info(ytest.shape)
            



In [16]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
    data_validation.train_test_spliting()

except Exception as e:
    raise e

[2023-10-01 12:28:53,519: INFO: common: yaml file: D:\model deploy end to end\NLP-text_classification\config\config.yaml loaded successfully]
[2023-10-01 12:28:53,522: INFO: common: yaml file: D:\model deploy end to end\NLP-text_classification\params.yaml loaded successfully]
[2023-10-01 12:28:53,524: INFO: common: yaml file: D:\model deploy end to end\NLP-text_classification\schema.yaml loaded successfully]
[2023-10-01 12:28:53,527: INFO: common: created directory at: artifacts]
[2023-10-01 12:28:53,531: INFO: common: created directory at: artifacts/data_validation]
[2023-10-01 12:28:53,553: INFO: 2337993418: True]
[2023-10-01 12:28:53,635: INFO: 2337993418: Splited data into training and test sets]
[2023-10-01 12:28:53,635: INFO: 2337993418: (4176, 1)]
[2023-10-01 12:28:53,635: INFO: 2337993418: (1392, 1)]
[2023-10-01 12:28:53,648: INFO: 2337993418: (4176,)]
[2023-10-01 12:28:53,651: INFO: 2337993418: (1392,)]
